In [9]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import FloatSlider, VBox, HBox, Dropdown, Layout, interactive_output

# Function to generate random points inside an ellipsoid
def generate_points(n_points, a, b, c):
    points = []
    while len(points) < n_points:
        x, y, z = np.random.uniform(-1, 1, 3)
        if x**2 + y**2 + z**2 <= 1:  # inside unit sphere
            points.append([a*x, b*y, c*z])
    return np.array(points)

# Rotation matrices
def rotation_matrix(angle, axis):
    c, s = np.cos(angle), np.sin(angle)
    if axis == 'x':
        return np.array([[1, 0, 0], [0, c, -s], [0, s, c]])
    elif axis == 'y':
        return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])
    elif axis == 'z':
        return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])

# Function to normalize coordinates to [-1, 1]
def normalize_coords(coords):
    max_val = np.abs(coords).max()
    if max_val > 0:
        return coords / max_val
    return coords

# Function to plot ellipsoid + points
def plot_ellipsoid(a=2, b=1, c=1.5, opacity=0.2, angle_deg=0.0, axis='z'):
    # Generate points
    pts = generate_points(500, a, b, c)

    # Convert angle to radians and apply rotation
    angle = np.deg2rad(angle_deg)
    R = rotation_matrix(angle, axis)
    pts_rot = pts @ R.T

    # Ellipsoid surface
    u = np.linspace(0, 2*np.pi, 50)
    v = np.linspace(0, np.pi, 25)
    x = a * np.outer(np.cos(u), np.sin(v))
    y = b * np.outer(np.sin(u), np.sin(v))
    z = c * np.outer(np.ones_like(u), np.cos(v))
    ellipsoid = np.stack([x.ravel(), y.ravel(), z.ravel()], axis=1)
    ellipsoid_rot = ellipsoid @ R.T

    # Normalize both points and ellipsoid
    all_coords = np.vstack([pts_rot, ellipsoid_rot])
    scale = np.abs(all_coords).max()
    if scale > 0:
        pts_rot /= scale
        ellipsoid_rot /= scale

    # Reshape back
    X, Y, Z = [ellipsoid_rot[:, i].reshape(x.shape) for i in range(3)]

    # Create figure
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=pts_rot[:,0], y=pts_rot[:,1], z=pts_rot[:,2],
        mode='markers', marker=dict(size=3, color='blue'),
        name='Points'))

    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        opacity=opacity,
        colorscale='Reds', showscale=False,
        name='Ellipsoid'))

    fig.update_layout(scene=dict(
        aspectmode='cube',
        xaxis=dict(range=[-1,1]),
        yaxis=dict(range=[-1,1]),
        zaxis=dict(range=[-1,1])
    ))
    fig.show()

# Custom sliders
controls = VBox([
    FloatSlider(min=0.5, max=3, step=0.1, value=2, description='a'),
    FloatSlider(min=0.5, max=3, step=0.1, value=1, description='b'),
    FloatSlider(min=0.5, max=3, step=0.1, value=1.5, description='c'),
    FloatSlider(min=0.1, max=1, step=0.1, value=0.3, description='Opacity'),
    FloatSlider(min=0, max=90, step=1, value=0, description='Angle (°)'),
    Dropdown(options=['x','y','z'], value='z', description='Axis')
], layout=Layout(width='300px'))

# Link sliders to plot function
out = interactive_output(plot_ellipsoid, {
    'a': controls.children[0],
    'b': controls.children[1],
    'c': controls.children[2],
    'opacity': controls.children[3],
    'angle_deg': controls.children[4],
    'axis': controls.children[5]
})

# Final layout: plot left, sliders right
display(HBox([out, controls]))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import FloatSlider, VBox, HBox, interactive_output, HTML

np.random.seed(42)
n = 400

# --------------------------
# Data generator
# --------------------------
def generate_data(var_x=5.0, var_y=2.0, var_z=1.0, rot_x=0, rot_y=0, rot_z=0):
    """Generate rotated 3D Gaussian ellipsoid data."""
    u = np.sqrt(var_x) * np.random.randn(n)
    v = np.sqrt(var_y) * np.random.randn(n)
    w = np.sqrt(var_z) * np.random.randn(n)
    data = np.vstack([u, v, w]).T

    Rx = np.array([[1, 0, 0],
                   [0, np.cos(np.deg2rad(rot_x)), -np.sin(np.deg2rad(rot_x))],
                   [0, np.sin(np.deg2rad(rot_x)),  np.cos(np.deg2rad(rot_x))]])
    Ry = np.array([[ np.cos(np.deg2rad(rot_y)), 0, np.sin(np.deg2rad(rot_y))],
                   [0, 1, 0],
                   [-np.sin(np.deg2rad(rot_y)), 0, np.cos(np.deg2rad(rot_y))]])
    Rz = np.array([[np.cos(np.deg2rad(rot_z)), -np.sin(np.deg2rad(rot_z)), 0],
                   [np.sin(np.deg2rad(rot_z)),  np.cos(np.deg2rad(rot_z)), 0],
                   [0, 0, 1]])
    R = Rz @ Ry @ Rx
    return data @ R.T

# --------------------------
# Euler axes & projections
# --------------------------
def project_to_plane(data, axis1, axis2):
    """Project data onto a 2D plane spanned by axis1, axis2."""
    coords1 = data @ axis1
    coords2 = data @ axis2
    return coords1, coords2

def euler_axes(alpha, beta, gamma):
    """Return three orthogonal axes based on Euler rotations (X->Y->Z order)."""
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(np.deg2rad(alpha)), -np.sin(np.deg2rad(alpha))],
                   [0, np.sin(np.deg2rad(alpha)),  np.cos(np.deg2rad(alpha))]])
    Ry = np.array([[ np.cos(np.deg2rad(beta)), 0, np.sin(np.deg2rad(beta))],
                   [0, 1, 0],
                   [-np.sin(np.deg2rad(beta)), 0, np.cos(np.deg2rad(beta))]])
    Rz = np.array([[np.cos(np.deg2rad(gamma)), -np.sin(np.deg2rad(gamma)), 0],
                   [np.sin(np.deg2rad(gamma)),  np.cos(np.deg2rad(gamma)), 0],
                   [0, 0, 1]])
    R = Rz @ Ry @ Rx
    return R[:,0], R[:,1], R[:,2]  # three orthogonal axes

# --------------------------
# Figure A: 3D + Cartesian projections
# --------------------------
def figA_3d_and_cartesian(var_x=5.0, var_y=2.0, var_z=1.0, rot_x=0, rot_y=0, rot_z=0):
    data = generate_data(var_x, var_y, var_z, rot_x, rot_y, rot_z)
    X, Y, Z = data[:,0], data[:,1], data[:,2]
    L = np.max(np.abs(data)) * 1.2

    # 3D plotly scatter
    fig3d = go.Figure(data=[go.Scatter3d(
        x=X, y=Y, z=Z, mode='markers',
        marker=dict(size=3, color='blue', opacity=0.6)
    )])
    fig3d.update_layout(
        scene=dict(
            xaxis=dict(range=[-L, L], title="X"),
            yaxis=dict(range=[-L, L], title="Y"),
            zaxis=dict(range=[-L, L], title="Z"),
            aspectmode='cube'
        ),
        title="3D Scatter in (X,Y,Z)"
    )
    fig3d.show()

    # 2D Cartesian projections (XY, YZ, ZX)
    fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))
    axes[0].scatter(X, Y, alpha=0.5, c="blue", s=10)
    axes[0].set_xlim(-L, L); axes[0].set_ylim(-L, L)
    axes[0].set_aspect("equal"); axes[0].set_title("XY")

    axes[1].scatter(Y, Z, alpha=0.5, c="blue", s=10)
    axes[1].set_xlim(-L, L); axes[1].set_ylim(-L, L)
    axes[1].set_aspect("equal"); axes[1].set_title("YZ")

    axes[2].scatter(Z, X, alpha=0.5, c="blue", s=10)
    axes[2].set_xlim(-L, L); axes[2].set_ylim(-L, L)
    axes[2].set_aspect("equal"); axes[2].set_title("ZX")

    plt.tight_layout()
    plt.show()

# --------------------------
# Figure B: Euler projections (α–β, β–γ, γ–α)
# --------------------------
def figB_euler_projections(var_x=5.0, var_y=2.0, var_z=1.0, rot_x=0, rot_y=0, rot_z=0,
                           alpha=0, beta=0, gamma=0):
    data = generate_data(var_x, var_y, var_z, rot_x, rot_y, rot_z)
    L = np.max(np.abs(data)) * 1.2
    ax_alpha, ax_beta, ax_gamma = euler_axes(alpha, beta, gamma)

    fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

    # α–β
    u, v = project_to_plane(data, ax_alpha, ax_beta)
    axes[0].scatter(u, v, alpha=0.5, c="red", s=10)
    axes[0].set_xlim(-L, L); axes[0].set_ylim(-L, L)
    axes[0].set_aspect("equal"); axes[0].set_title("α–β")

    # β–γ
    u, v = project_to_plane(data, ax_beta, ax_gamma)
    axes[1].scatter(u, v, alpha=0.5, c="red", s=10)
    axes[1].set_xlim(-L, L); axes[1].set_ylim(-L, L)
    axes[1].set_aspect("equal"); axes[1].set_title("β–γ")

    # γ–α
    u, v = project_to_plane(data, ax_gamma, ax_alpha)
    axes[2].scatter(u, v, alpha=0.5, c="red", s=10)
    axes[2].set_xlim(-L, L); axes[2].set_ylim(-L, L)
    axes[2].set_aspect("equal"); axes[2].set_title("γ–α")

    plt.tight_layout()
    plt.show()

# --------------------------
# NEW: Figure C — 3D in (α, β, γ) coordinates
# --------------------------
def figC_alpha_beta_gamma(var_x=5.0, var_y=2.0, var_z=1.0, rot_x=0, rot_y=0, rot_z=0,
                          alpha=0, beta=0, gamma=0):
    """
    Plot the same data expressed in the (α, β, γ) coordinate system.
    This is just a basis change: coords = data @ [ax_α ax_β ax_γ]
    """
    data = generate_data(var_x, var_y, var_z, rot_x, rot_y, rot_z)
    ax_alpha, ax_beta, ax_gamma = euler_axes(alpha, beta, gamma)
    A = np.column_stack([ax_alpha, ax_beta, ax_gamma])   # basis matrix
    coords = data @ A                                    # (α, β, γ) coordinates

    L = np.max(np.abs(coords)) * 1.2

    fig3d = go.Figure(data=[go.Scatter3d(
        x=coords[:,0], y=coords[:,1], z=coords[:,2],
        mode='markers',
        marker=dict(size=3, color='purple', opacity=0.6)
    )])
    fig3d.update_layout(
        scene=dict(
            xaxis=dict(range=[-L, L], title="α"),
            yaxis=dict(range=[-L, L], title="β"),
            zaxis=dict(range=[-L, L], title="γ"),
            aspectmode='cube'
        ),
        title="3D Scatter in (α, β, γ)"
    )
    fig3d.show()

# --------------------------
# Sliders
# --------------------------
varx_slider = FloatSlider(min=0.1, max=10, step=0.1, value=5, description="Var X")
vary_slider = FloatSlider(min=0.1, max=10, step=0.1, value=2, description="Var Y")
varz_slider = FloatSlider(min=0.1, max=10, step=0.1, value=1, description="Var Z")

rotx_slider = FloatSlider(min=0, max=180, step=5, value=20, description="Rot X")
roty_slider = FloatSlider(min=0, max=180, step=5, value=10, description="Rot Y")
rotz_slider = FloatSlider(min=0, max=180, step=5, value=0,  description="Rot Z")

alpha_slider = FloatSlider(min=0, max=180, step=5, value=0, description="α")
beta_slider  = FloatSlider(min=0, max=180, step=5, value=0, description="β")
gamma_slider = FloatSlider(min=0, max=180, step=5, value=0, description="γ")

# --------------------------
# Interactive outputs
# --------------------------
out_figA = interactive_output(
    figA_3d_and_cartesian,
    {
        "var_x": varx_slider, "var_y": vary_slider, "var_z": varz_slider,
        "rot_x": rotx_slider, "rot_y": roty_slider, "rot_z": rotz_slider
    }
)

out_figB = interactive_output(
    figB_euler_projections,
    {
        "var_x": varx_slider, "var_y": vary_slider, "var_z": varz_slider,
        "rot_x": rotx_slider, "rot_y": roty_slider, "rot_z": rotz_slider,
        "alpha": alpha_slider, "beta": beta_slider, "gamma": gamma_slider
    }
)

# NEW: Figure C output
out_figC = interactive_output(
    figC_alpha_beta_gamma,
    {
        "var_x": varx_slider, "var_y": vary_slider, "var_z": varz_slider,
        "rot_x": rotx_slider, "rot_y": roty_slider, "rot_z": rotz_slider,
        "alpha": alpha_slider, "beta": beta_slider, "gamma": gamma_slider
    }
)

# --------------------------
# Layouts
# --------------------------
titleA = HTML("<h3>Figure A — 3D Scatter + Cartesian Projections (XY, YZ, ZX)</h3>")
uiA = VBox([
    titleA,
    HBox([varx_slider, vary_slider, varz_slider]),
    HBox([rotx_slider, roty_slider, rotz_slider]),
    out_figA
])

titleB = HTML("<h3>Figure B — Euler Projections (α–β, β–γ, γ–α)</h3>")
uiB = VBox([
    titleB,
    HBox([alpha_slider, beta_slider, gamma_slider]),
    out_figB
])

titleC = HTML("<h3>Figure C — 3D Scatter in (α, β, γ) Coordinates</h3>")
uiC = VBox([
    titleC,
    HBox([alpha_slider, beta_slider, gamma_slider]),
    out_figC
])

# Display all three
VBox([uiA, uiB, uiC])
